# Make clean data

In [11]:
import pandas as pd

bubbleList = []
countingList = []
misPredList = []
mergeList = []
quickList = []
readCSVList = []
pcList = ["6", "16", "21", "23"]

# quickList.append(pd.read_csv("SummerResults/quickSort-400runs-PC16.csv"))
# quickList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

# bubbleList.append(pd.read_csv("SummerResults/bubbleSort-400runs-PC16.csv"))
# bubbleList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

# readCSVList.append(pd.read_csv("SummerResults/readCSV-PC16.csv"))
# readCSVList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

misPredList.append(pd.read_csv("SummerResults/bubbleSort-branchMisPredict-PC16.csv"))
misPredList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]




In [12]:
def removeNonsense(df):
    df = df[df["PKG (J)"]>=0]
    df = df[df["CPU (J)"]>=0]
    df = df[df["Time (ms)"]>=0]
    return df

In [13]:
def makeinputColumn(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[0])

    df["Input case"] = l

def makeSizecolumn(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[1]))

    df["Size"] = l

In [14]:
def makeinputColumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[1].lower())
    df["Input case"] = l

def makeSizecolumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[-1]))

    df["Size"] = l

In [15]:
def removeOutliers(df, type_, size, col):
    q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
    q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
    IQR = q3 - q1 
    LF = q1 - 1.5 * IQR
    UF = q3 + 1.5 * IQR

    df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
    df2 = df1[df1["Input case"] == type_][df1["Size"]==size][df1[col] >= LF]
    return df2[col]


In [16]:
def CleanDataframe(df, DataOrBase):
    if DataOrBase == "data":
        cases = ["best", "random", "worst"]
    elif DataOrBase == "misPred":
        cases = ["greaterThan0", "greaterThanEquals0"]
    else:
        cases = ["random", "reversesorted", "sorted"]

    cleanList = []
    for case in cases:
        if case == "greaterThanEquals0" or case == "greaterThan0":
            for size in [10000,20000,30000,40000,50000]:
                cleanPortion = pd.DataFrame({"PKG (J)":[]})
                for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                    cleanPortion[col] = removeOutliers(df, case, size, col)
                
                cleanPortion["Input case"] = [case for _ in range(len(cleanPortion))]
                cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]

                cleanList.append(cleanPortion)
        elif case != "random" and case != "countingrandom":
            for size in [25000,50000,75000,100000,125000,150000,175000,200000]:
                cleanPortion = pd.DataFrame({"PKG (J)":[]})
                for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                    cleanPortion[col] = removeOutliers(df, case, size, col)
                
                cleanPortion["Input case"] = [case for _ in range(len(cleanPortion))]
                cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]

                cleanList.append(cleanPortion)
        else:
            for i in range(1,11):
                for size in [25000,50000,75000,100000,125000,150000,175000,200000]:
                    cleanPortion = pd.DataFrame({"PKG (J)":[]})
                    for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                        cleanPortion[col] = removeOutliers(df, case+str(i), size, col)
                
                    cleanPortion["Input case"] = [case+str(i) for _ in range(len(cleanPortion))]
                    cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]
                    cleanList.append(cleanPortion)
    cleanBubble = pd.concat(cleanList)
    return cleanBubble

In [17]:
def prepareDataframe(df, DataOrBase):
    df = removeNonsense(df)

    if DataOrBase == "data":
        makeinputColumn(df)
        makeSizecolumn(df)
    else:
        makeinputColumn_readCSV(df)
        makeSizecolumn_readCSV(df)    

    return df


In [18]:
def CleanCSV (df, DataOrBase, name, pcNO):
    ready = prepareDataframe(df, DataOrBase)
    clean = CleanDataframe(ready, DataOrBase).dropna().reset_index(drop=True)
    display(clean)
    clean.to_csv("FinalAnalysis\\CleanData\\"+name+"Clean-PC"+pcNO+".csv", sep=",", index=False)

In [19]:

#CleanCSV(quickList[0], "data","QucikSlow-400runs",pcList[1])
# CleanCSV(readCSVList[0], "base","readCSVSlow",pcList[1])

#CleanCSV(bubbleList[0], "data","Bubble-400runs",pcList[1])

CleanCSV(misPredList[0], "misPred","BranchMisPrediction",pcList[1])

IndexError: list index out of range

In [ ]:

#CleanCSV(bubbleList[0], "data","Bubble",pcList[0])
